In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
target_x_sample = pd.read_csv(PATH["target_x_sample.tsv"], sep="\t", index_col=0)

In [ ]:
annotation = (("Peek", SETTING["gene_sets_to_peek"], 12, "#20d9ba"),)

In [ ]:
for target_name in target_x_sample.index:

    output_directory_path = "{}/{}".format(
        PATH["compare_differentially_expressed_gene_set/"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    score_moe_p_value_fdr = pd.read_csv(
        "{}/{}/all.tsv".format(
            PATH["find_differentially_expressed_gene_set/"], target_name
        ),
        sep="\t",
        index_col=0,
    )

    if not score_moe_p_value_fdr["P-Value"].isna().any():

        x = score_moe_p_value_fdr["Score"]

        ccal.plot_point_and_annotate(
            x,
            pd.Series(1 - score_moe_p_value_fdr["P-Value"], name="Significance"),
            "x",
            annotation=annotation,
            title=dict(text="All"),
            html_file_path="{}/all.html".format(output_directory_path),
        )

        for gene_set_file_path in SETTING["gene_set_file_paths"]:

            gene_sets_name = gene_set_file_path.split("/")[-1]

            gene_sets = (
                score_moe_p_value_fdr.index & ccal.read_gmt(gene_set_file_path).index
            )

            x = x[gene_sets]

            if not x.isna().all():

                ccal.plot_point_and_annotate(
                    x,
                    pd.Series(
                        1 - score_moe_p_value_fdr["P-Value"], name="Significance"
                    )[gene_sets],
                    "x",
                    annotation=annotation,
                    title=dict(text=gene_sets_name),
                    html_file_path="{}/{}.html".format(
                        output_directory_path, gene_sets_name
                    ),
                )